In [2]:
import requests

myQuery = 'Leigh M. Rothschild'
rows = 1000
rqstUrl = f'https://assignment-api.uspto.gov/patent/basicSearch?rows={rows}&query={myQuery}'
print(rqstUrl)
rsp = requests.get(rqstUrl)
print(rsp)

https://assignment-api.uspto.gov/patent/basicSearch?rows=1000&query=Leigh M. Rothschild
<Response [200]>


In [4]:
from lxml import etree

xml_ascii = rsp.text.encode('ascii')
rsproot = etree.fromstring(xml_ascii)
#rsproot = etree.parse('fake.xml')
rslt = rsproot.xpath('result')[0]

def spliceAssigneeAddress(addy_tuple):
    result = ''
    for adt in addy_tuple[1:]:
        if ('NULL' != adt):
            result = f'{result}{adt} '
    return result.strip()

def parseDoc(d):
    '''
    Extract what we want from a <doc> Element and return it as a dict.
    '''
    result = {}
    
    result['last_updated'] = d.xpath('.//date[@name="lastUpdateDate"]/text()')[0]
    result['execution_date'] = d.xpath('.//date[@name="patAssignorEarliestExDate"]/text()')[0]
    result['assignor_names'] = d.xpath('.//arr[@name="patAssignorName"]/str/text()')

    result['invention_title'] =  d.xpath('.//arr[@name="inventionTitle"]/str/text()')
    result['inventors'] =  d.xpath('.//arr[@name="inventors"]/str/text()')
    result['application_number'] = d.xpath('.//arr[@name="applNum"]/str/text()')
    result['filing_date'] =  d.xpath('.//arr[@name="filingDate"]/date/text()')
    result['publication_number'] =  d.xpath('.//arr[@name="publNum"]/str/text()')
    result['publication_date'] =  d.xpath('.//arr[@name="publDate"]/date/text()')
    result['patent_number'] = d.xpath('.//arr[@name="patNum"]/str/text()')
    result['issue_date'] =  d.xpath('.//arr[@name="issueDate"]/date/text()')
    result['pct_number'] =  d.xpath('.//arr[@name="pctNum"]/str/text()')
    result['intl_publication_date'] =  d.xpath('.//arr[@name="intlPublDate"]/date/text()')
    
    correspondent_name = d.xpath('.//str[@name="corrName"]/text()')[0]
    correspondent_address_1 = d.xpath('.//str[@name="corrAddress1"]/text()')
    correspondent_address_2 = d.xpath('.//str[@name="corrAddress2"]/text()')
    correspondent_address_3 = d.xpath('.//str[@name="corrAddress3"]/text()')
    correspondent_address = ''
    for i in (correspondent_address_1, correspondent_address_2, correspondent_address_3):
        if ('NULL' != i):
            try:
                correspondent_address = f'{correspondent_address}{i[0]} '
            except IndexError:
                continue
    correspondent_address = correspondent_address[:-1]
    result['correspondent'] = (correspondent_name, correspondent_address)

    assignee_names = d.xpath('.//arr[@name="patAssigneeName"]/str/text()')
    assignee_address_1 = d.xpath('.//arr[@name="patAssigneeAddress1"]/str/text()')
    assignee_address_2 = d.xpath('.//arr[@name="patAssigneeAddress2"]/str/text()')
    assignee_city = d.xpath('.//arr[@name="patAssigneeCity"]/str/text()') 
    assignee_state = d.xpath('.//arr[@name="patAssigneeState"]/str/text()') 
    assignee_country = d.xpath('.//arr[@name="patAssigneeCountryName"]/str/text()') 
    assignee_postcode = d.xpath('.//arr[@name="patAssigneePostcode"]/str/text()') 
    zipped_assignee = list(zip(assignee_names, assignee_address_1, assignee_address_2, assignee_city, assignee_state, assignee_country, assignee_postcode))
    assignees = []
    for za in zipped_assignee:
        assignees.append((za[0], spliceAssigneeAddress(za)))
    result['assignees'] = assignees

    return result
    
doc_dicts = []
# Find and parse all 'doc' tags
for d in rslt.iter('doc'):
    doc_dicts.append(parseDoc(d))
    # end for loop.
print(f'{len(doc_dicts)} doc_dicts created.')

150 doc_dicts created.


In [5]:
for d in doc_dicts:
    print(f'{d["correspondent"]}')

('SUGOURI BATRA, PATENT ASSET MANAGEMENT', '1 EAST BROWARD BLVD. SUITE 700 FORT LAUDERDALE, FL 33301')
('LEIGH M. ROTHSCHILD', '1574 QUAYSIDE TERRACE MIAMI, FL 33138')
('LEIGH M ROTHCHILD', '1801 NE 123 STREET SUITE 314 MIAMI, FL 33181')
('FERRAIUOLI LLC', '221 PONCE DE LEON AVE., 5TH FL SAN JUIAN, 00917 PUERTO RICO')
('EUGENIO J. TORRES', '221 PONCE DE LEON AVE 5TH FLOOR SAN JUAN, 00917 PUERTO RICO')
('FERRAIUOLI', '221, PONCE DE LEON AVE SUITE 500 SAN JUAN, 00917 PUERTO RICO')
('FERRAIUOLI LLC', '221 PONCE DE LEON AVE. 5TH FL SAN JUAN, 00917 PUERTO RICO')
('FERRAIUOLI LLC', '221 PONCE DE LEON AVE., 5TH FL SAN JUAN, 00917 PUERTO RICO')
('GREER, BURNS & CRAIN, LTD.', '300 S. WACKER DRIVE CHICAGO, IL 60606')
('GREER, BURNS & CRAIN, LTD.', '300 S. WACKER DRIVE CHICAGO, IL 60606')
('GREER, BURNS & CRAIN, LTD.', '300 SOUTH WACKER DRIVE CHICAGO, IL 60606')
('GREER, BURNS & CRAIN, LLC', '300 S. WACKER DRIVE CHICAGO, IL 60606')
('GREER, BURNS & CRAIN, LTD.', '300 SOUTH WACKER DRIVE SUITE 2500

In [6]:
for d in doc_dicts:
    print(f'{d["assignees"]}')

[('SRR PATENT HOLDINGS, LLC', '1712 PIONEER AVE. CHEYENNE WYOMING 82001')]
[('DRUG INFORMATION RETRIEVAL SYSTEM, LLC', '1801 NE 123RD STREET MIAMI FLORIDA 33181')]
[('ROTHSCHILD, LEIGH M', '1801 NE 123 STREET SUITE 314 MIAMI FLORIDA 33181')]
[('QMAGE, INC.', '12555 BISCAYNE BLVD #861 MIAMI FLORIDA 33138')]
[('ARIEL INVENTIONS, LLC', '1574 NE QUAYSIDE TERRACE MIAMI FLORIDA 33138')]
[('QMAGE, LLC', '1400 PRESTON ROAD SUITE 400 PLANO TEXAS 75093')]
[('SYMBOLOGY INNOVATIONS, LLC', '1400 PRESTON RD. STE. 400 PLANO TEXAS 75093')]
[('ROTHSCHILD PATENT IMAGING, LLC', '1574 NE QUAYSIDE TERRACE MIAMI FLORIDA 33136')]
[('ROTHSCHILD PATENT IMAGING, LLC', '1574 NE QUAYSIDE TERRACE MIAMI FLORIDA 33136')]
[('REAGAN INVENTIONS, LLC', '19333 COLLINS AVENUYE, #2501 SUNNY ISLES BEACH FLORIDA 33160')]
[('REAGAN INVENTIONS, LLC', '19333 COLLINS AVENUE, #2501 SUNNY ISLES BEACH FLORIDA 33160')]
[('ROTHSCHILD PATENT IMAGING, LLC', '1574 NE QUAYSIDE TERRACE MIAMI FLORIDA 33136')]
[('ROTHSCHILD AUTOMOTIVE TECHN